### ResNet

케밍 헤 (He)... 연봉 이 100억이 넘는대... ㅅㅂ 난 1억 받기도 힘들다... 

Residual을 이용해서 학습을 했다. 깊으면 깊을수록 모델이 좋아야 햇는데, 그렇지가 않았다.

결국엔 최적화의 문제로 풀었다. 깊은 모델은 학습이 더 어렵다.

---

 - 최적화 잘된 swallow 모델을 가져다가, 깊은 모델을 만드는 시도를 했다. -> 그래서 만든것이 ResNet이다.
 - Residual블럭을 계속 쌓았다.
 
### 배치노말라이제이션 하는 이유 -> 학습 수렴이 잘 된다.

### weight decay -> 패널티다. 커널 레귤라이저.

### 드롭아웃은 오버피팅을 늦출수는있지만, 결국막지는 못한다.

클래시피케이션은 에러값이 사람보다 더 작다. 그래서 더이상 이 분야의 이미지 넷은 없어졌다. 

 - vgg16은 메모리 줄이는 애들이 없는데, 구글넷은 1x1컨볼루션 등 차원줄여주는 애들이 있다.
 - ResNet은 언제 1바이1하고 있었을까? -> 수많은 노가다를 통해만들었다.

In [1]:
from tensorflow.keras.applications import InceptionResNetV2, InceptionV3

In [2]:
model1 = InceptionResNetV2()
model2 = InceptionV3()

In [3]:
model1.summary()


Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
________________________________________________________________________________

Total params: 55,873,736
Trainable params: 55,813,192
Non-trainable params: 60,544
__________________________________________________________________________________________________


In [4]:
model2.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 149, 149, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 149, 149, 32) 0           batch_normalization_203[0][0]    
_______________________________________________________________________________________

### R CNN

https://github.com/broadinstitute/keras-rcnn/blob/master/keras_rcnn/models/_rcnn.pym

 - 로컬라이제이션 + 클래시피케이션이 디텍션이다.
 - 어떤 크기, 비율인지 학습을 통해서 알 수 가 있다. 위치를 일종의 regression을 통해 찾아낼 수 있는 것이다. 이것이 바로 디텍션이다.

### Bounding Box Regression

RoI pooling: 항상 이미지크기와 상관없이 크기가 똑같이 나온다는게 핵심이다. 항상 등분을 똑같이 해주면 해결된다고 생각해라. ★★★

https://login.manning.com/login

사이트 들어가면, 좋은 책들이 많다.

In [12]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions, VGG16

In [6]:
x = load_img('C:/aaa.png', target_size = (224, 224))
img_x = img_to_array(x)
img_x

array([[[112., 129., 127.],
        [115., 133., 130.],
        [116., 134., 130.],
        ...,
        [196., 198., 197.],
        [196., 196., 196.],
        [196., 198., 197.]],

       [[107., 120., 121.],
        [103., 120., 118.],
        [108., 122., 119.],
        ...,
        [196., 198., 197.],
        [197., 199., 198.],
        [195., 197., 196.]],

       [[108., 113., 120.],
        [101., 107., 111.],
        [106., 116., 115.],
        ...,
        [196., 198., 197.],
        [196., 197., 198.],
        [195., 197., 196.]],

       ...,

       [[120.,  91.,  57.],
        [119.,  91.,  57.],
        [119.,  94.,  58.],
        ...,
        [173., 161., 163.],
        [126., 107., 108.],
        [ 95.,  74.,  73.]],

       [[119.,  93.,  59.],
        [117.,  91.,  56.],
        [118.,  94.,  58.],
        ...,
        [191., 194., 195.],
        [193., 194., 193.],
        [197., 185., 185.]],

       [[118.,  90.,  58.],
        [117.,  91.,  58.],
        [118.,  

In [13]:
model = VGG16(include_top = False)

In [7]:
import numpy as np
t = preprocess_input(img_x)
t = t[np.newaxis]

In [15]:
a = model.predict(t)
a.shape

(1, 7, 7, 512)

In [40]:
train = a.reshape(1, -1)
train2 = np.fliplr(a[0])
test = np.array([0, 1])
te = train2[np.newaxis]

train = np.concatenate([a, te]).reshape(2, -1)

In [21]:
te.shape

(1, 7, 7, 512)

In [41]:
train.shape

(2, 25088)

In [34]:
test.shape

(2,)

In [43]:
train.shape # 예측하는 데이터가 2차원이 되어야 한다.

(2, 25088)

In [29]:
from sklearn.svm import SVC

svm = SVC()

In [42]:
svm.fit(train, test)

C:\Users\white\Anaconda3\envs\tf2.0-gpu\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)